In [44]:
# Загрузка документов
import os
from langchain_community.document_loaders import SitemapLoader, RecursiveUrlLoader

os.environ["USER_AGENT"] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)"

ROOT_URL = "https://antarcticwallet.com"
SITEMAP_URL = f"{ROOT_URL}/sitemap.xml"

# # 1) Загружаем все страницы из sitemap
# sitemap_loader = SitemapLoader(
#     web_path=SITEMAP_URL,
#     filter_urls=[ROOT_URL],  # на всякий случай ограничиваем доменом
# )
# sitemap_docs = sitemap_loader.load()

# 2) Дополнительно рекурсивно обходим сайт от корня
recursive_loader = RecursiveUrlLoader(
    url=ROOT_URL,
    max_depth=2,          # глубину при желании можно увеличить
    prevent_outside=True  # не выходим за пределы домена
)
recursive_docs = recursive_loader.load()

# 3) Объединяем всё в один список документов для RAG
# docs = sitemap_docs + recursive_docs
docs = recursive_docs

print(f"Total documents: {len(docs)}")
print(f"Total characters: {sum(len(doc.page_content) for doc in docs)}")

# 1
# Total documents: 1
# Total characters: 987822

# 2
# Total documents: 16
# Total characters: 7040675

# 3
# Total documents: 16
# Total characters: 4124981

Total documents: 16
Total characters: 5676661


In [45]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.HTML,   # учитываем структуру HTML
    chunk_size=1200,          # немного больше, т.к. структура сохраняется лучше
    chunk_overlap=200,
)

splits = text_splitter.split_documents(docs)

In [46]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import Embeddings

# Базовая модель
base_embeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/ru-en-RoSBERTa"
)

class PrefixedEmbeddings(Embeddings):
    def __init__(self, base, query_prefix="", doc_prefix=""):
        self.base = base
        self.query_prefix = query_prefix
        self.doc_prefix = doc_prefix

    def embed_documents(self, texts):
        texts_prefixed = [self.doc_prefix + t for t in texts]
        return self.base.embed_documents(texts_prefixed)

    def embed_query(self, text):
        return self.base.embed_query(self.query_prefix + text)

embeddings = PrefixedEmbeddings(
    base_embeddings,
    query_prefix="search_query: ",
    doc_prefix="search_document: ",
)

Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [48]:
from pathlib import Path
from langchain_chroma import Chroma

persist_directory = "./chroma_db7"

if Path(persist_directory).exists():
    # Индекс уже есть — просто загружаем
    vectorstore = Chroma(
        embedding_function=embeddings,
        persist_directory=persist_directory,
    )
else:
    # Первый запуск — создаём индекс
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory=persist_directory,
    )

In [26]:
# retriever = vectorstore.as_retriever(
#     search_type="similarity_score_threshold",
#     search_kwargs={
#         "score_threshold": 0.3,
#         "k": 12,
#     },
# )

In [50]:
retriever = vectorstore.as_retriever(
    search_type="mmr",  # вместо простого similarity
    search_kwargs={
        "k": 8,          # сколько документов вернуть в итоге
        "fetch_k": 32,   # из скольких кандидатов выбирать (больше = разнообразнее)
        # при желании можно добавить lambda_mult для тонкой настройки
        # lambda_mult – это параметр MMR, который задаёт баланс между релевантностью документов запросу и их разнообразием: чем ближе значение к 1, тем сильнее приоритет близости к запросу, чем ближе к 0 – тем важнее разнообразие результатов.
        # "lambda_mult": 0.8,
    },
)

In [51]:
MAX_CHARS = 8000

def format_docs(docs):
    formatted = []
    total_len = 0

    for doc in docs:
        source = doc.metadata.get("source", "unknown_source")
        page = doc.metadata.get("page", None)

        header = f"Source: {source}"
        if page is not None:
            header += f" | Page: {page}"

        text = doc.page_content.strip()
        block = f"{header}\n{text}"

        # если следующий блок слишком раздует контекст — останавливаемся
        if total_len + len(block) > MAX_CHARS:
            break

        formatted.append(block)
        total_len += len(block)

    return "\n\n---\n\n".join(formatted)

In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Ты помощник, который отвечает СТРОГО НА РУССКОМ ЯЗЫКЕ. "
        "Используй только информацию из предоставленного контекста, не придумывай факты. "
        "Если ответа в контексте нет или данных недостаточно, честно скажи, что не нашёл ответа в базе. "
        "Всегда упоминай источник в формате из заголовка (Source и Page). "
        "Отвечай кратко и по делу, обычно до 5–7 предложений."
    ),
    # историю можно не заполнять, но структура уже есть
    MessagesPlaceholder("history"),
    (
        "human",
        "Контекст:\n{context}\n\nВопрос: {question}"
    ),
])

In [53]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    api_key="None",
    base_url="http://127.0.0.1:11434/v1",
    model="hf.co/bartowski/Mistral-Nemo-Instruct-2407-GGUF:Q4_K_M",

    # важные параметры для RAG
    temperature=0.2,      # меньше фантазии
    max_tokens=512,       # контролируем длину ответа
    top_p=0.8,
)

In [55]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def ensure_context(input_dict: dict) -> dict:
    """
    Если retriever не нашёл ничего полезного и контекст пустой,
    явно помечаем это в контексте, чтобы модель не фантазировала.
    """
    context = input_dict.get("context", "").strip()
    if not context:
        input_dict["context"] = (
            "Контекст пуст: ретривер не нашёл ни одного подходящего фрагмента. "
            "Если ответ важен, лучше явно сказать пользователю об этом."
        )
    return input_dict

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
        "history": lambda _: [],  # пока истории нет — передаём пустой список
    }
    | RunnableLambda(ensure_context)   # защита от пустого контекста
    | prompt
    | llm
    | StrOutputParser()
).with_config(run_name="rag_chain")

In [37]:
print(rag_chain.invoke("Что такое Antarctic Wallet?"))

Antarctic Wallet — это сервис для оплаты товаров и услуг с помощью криптовалют. Чтобы начать пользоваться им, нужно открыть кошелек через Telegram-бота @antarctic_wallet_bot или установить приложение на телефон через браузер в режиме PWA (Progressive Web App). Сервис соблюдает законодательство Российской Федерации и не нарушает статью 259 ФЗ РФ.


In [38]:
print(rag_chain.invoke("Как начать пользвоваться Antarctic Wallet?"))

Чтобы начать пользоваться Antarctic Wallet, следуйте инструкции:
1. Откройте Telegram (если нет аккаунта, зарегистрируйтесь)
2. Найдите официального бота по юзернейму: @antarctic_wallet_bot
3. Нажмите на кнопки «Начать» → «Открыть Wallet».

Если хотите установить приложение на телефон, воспользуйтесь функцией PWA (Progressive Web App):
1. Перейдите по адресу <https://app.antarcticwallet.com/> в браузере телефона
2. Нажмите на центральную кнопку «Поделиться» в нижнем меню браузера
3. Выберите функцию «На экран домой»
4. Авторизуйтесь через телеграм-бота (@antarctic_wallet_bot) при запуске приложения


In [56]:
print(rag_chain.invoke("Какие комиссии есть в Antarctic Wallet?"))

В Antarctic Wallet есть комиссии на пополнение. Для USDT в сети TRC20 комиссия составляет $2,75, для TON в сети TON — 0,2 TON, а для USDT в сети TON комиссия отсутствует (Source: https://antarcticwallet.com/faq, Page: Есть ли комиссии в Antarctic Wallet?).


In [57]:
print(rag_chain.invoke("Нужно ли проходить KYC?"))

Да, KYC (Know Your Customer) верификация является обязательной процедурой для криптовалютных платформ и сервисов, в том числе для доступа к основному функционалу Antarctic Wallet. Ее цель — подтвердить личность пользователя для соблюдения норм AML (Anti-Money Laundering, противодействие отмыванию денег). Прохождение KYC верификации в Antarctic Wallet полностью безопасно, так как сервис не хранит данные своих пользователей и работает с KYC провайдером — Sumsub, известная своими крупными клиентами: Bybit, Binance, Bingx и др. (Source: https://antarcticwallet.com)
